# Why PyEPICS?

* Easy to learn
* Highly readable syntax
* Powerful features
* Widely used at many research facilities

### PyEPICS enables you to...

* Quickly access data from EPICS PVs in convenient formats (eg numpy arrays)
* Easily create Python representations of physical devices
* Automate interactions with multiple devices
* Avoid the low-level details of the channel access protocol and focus on your application

# Setup

## Requirements

Install Miniconda Python 3.5 and create a `python-workshop` conda environment as described in the [Installing Python](1_Installing_Python.ipynb) notebook.

## Installing PyEPICS

### Windows

```
activate python-workshop
pip install pyepics
```

### Linux

```
source activate python-workshop
conda install -c lightsource2 pyepics readline=6.2.5
export PATH="/opt/miniconda3/envs/python-workshop/epics/bin/linux-x86_64:$PATH"
```

### macOS

1. Install homebrew
2. Download and build EPICS base v3.14.12.5
3. Add `export EPICS_BASE=/epics/base` and `export EPICS_HOST_ARCH=darwin-x86` to your `~/.bash_profile`
4. 

  ```
  source activate python-workshop
  pip install pyepics
  ```

## Getting started

Once you have pyepics installed, install the following packages we will use for this workshop:

```
# make sure you have the python-workshop conda environment enabled
conda install jupyter matplotlib numpy
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

Run `jupyter notebook` in the folder containing this notebook.

# Let's get some PVs

## Load PyEPICS


To load PyEPICS, simply **`import epics`**:

In [ ]:
import epics

If all went well you should be able to fetch the beam current with `epics.caget`:

In [ ]:
epics.caget('SR11BCM01:CURRENT_MONITOR')

For this workshop we have created a special IOC generator that will launch a private IOC with some
PVs you can poke without worrying about doing any damage to real equipment.

Go to http://pythonworkshop.staff.synchrotron.org.au and click the **Launch IOC** button.

Copy the prefix at the top of the screen and assign it to a variable in the next cell:

In [ ]:
PREFIX = 'GIZMO-20T'  # Replace this with your IOC's prefix

## Creating a PV object

Although you can get and set PV values with `caget` and `caput`, PyEPICS offers a more powerful interface
that you should always use: `PV` objects. As well as supporting accessing the value of the PV, this interface
enables you to access metadata like the units and alarm levels.

To create a `PV` variable, import the `PV` class and call it supplying the name of the pv you wish to connect to:

In [ ]:
from epics import PV

In [ ]:
temperature_pv = PV(PREFIX + ':TEMPERATURE')

With `PV` objects you can fetch the values with `.value` or `.get()`:

In [ ]:
temperature_pv.value

In [ ]:
temperature_pv.get()

## Challenge

Try getting some of the other PVs listed on `http://pythonworkshop.staff.synchrotron.org.au/iocs/<your-ioc-prefix>`. Don't be surpised if you get unexpected output from the `:LONG_STRING` PV - that will be explained in due time.

## Other PV properties

As well as the value, PV objects allow you to access the following attributes on the PV object:

* `pv.value`
* `pv.char_value`
* `pv.status`
* `pv.type`
* `pv.ftype`
* `pv.host`
* `pv.count`
* `pv.nelm`
* `pv.read_access` / `pv.write_access` / `pv.access`
* `pv.severity`
* `pv.timestamp`
* `pv.precision`
* `pv.units`
* `pv.enum_strs`
* `pv.info`
* `pv.upper_disp_limit` / `pv.lower_disp_limit`
* `pv.upper_alarm_limit` / `pv.lower_alarm_limit`
* `pv.upper_warning_limit` / `pv.lower_warning_limit`
* `pv.upper_ctrl_limit` / `pv.lower_ctrl_limit`
* `pv.put_complete`
* `pv.callbacks`
* `pv.connection_callbacks`

These are described in the PyEPICS documentation: http://cars9.uchicago.edu/software/python/pyepics3/pv.html#attributes

## Challenge

1. See if you can determine the units, alarm limits and severity of the temperature PV.
2. The severity is an integer with the following meaning:

    ```
    0 = NO_ALARM
    1 = MINOR
    2 = MAJOR
    3 = INVALID
    ```

    Write a function that returns a human readable severity for a given PV.

# Getting Stringy PVs

EPICS stores strings in several different ways which need to be handled differently in pyepics.

* Strings
* Character arrays
* Enums labels

Strings are easy to work with - you can use the `.value` attribute or `.get()` method:

In [ ]:
short_string_pv = PV(PREFIX + ':SHORT_STRING')

In [ ]:
short_string_pv.value

In [ ]:
short_string_pv.get()

But now try getting the value of the PV ending in `:LONG_STRING`

In [ ]:
long_string_pv = PV(PREFIX + ':LONG_STRING')

In [ ]:
long_string_pv.value

Not what we expected! This is because this PV is actually an array of bytes and PyEPICS can't be sure that
data is meant to represent a string rather than data from an image, for example.

Fortunately PyEPICS makes it easy for us to tell it to get it as a string. Either...

1. Use the `.char_value` attribute
2. Use the `.get()` method and provide an argument of `as_string=True`

### Challenge:

Try both methods to get the text stored in the `:LONG_STRING` PV.

The final time you might want a string representation of a PV is for enumeration records that have a number of states.

The `:ALERT` PV is an enum... if we ask for it's value we get the index of the state it is in:

In [ ]:
alert_pv = PV(PREFIX + ':ALERT')

alert_pv.value

### Challenge:

1. Get the state of the `alert_pv` as a string.
2. Use the `.enum_strs` attribute to find out all possible states.

# Arrays and waveforms

When you get a waveform PV it is returned as a numpy array:

In [ ]:
wave_pv = PV(PREFIX + ':WAVE')
wave_pv.value[:10]

We will be covering `numpy` in more detail in the next workshop but for now, try calling
`.min()`, `.max()`, `.mean()`, `.std()` on the `wave_pv.value`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

### Challenge

Use `plt.plot()` to plot the data from the `:WAVE` PV.

## What happens if a PV doesn't exist

PyEPICS has a shortcoming that if you ask for the value of a PV that doesn't exist (eg if the IOC has gone offline), it doesn't raise
an error but just returns "`None`".

In [ ]:
invalid_pv = PV('INVALID_NAME')
value = invalid_pv.get()
print(value)

This means you may need to check the value you get back.

### Challenge:

Use an `if` statement to detect whether the value returned from a PV of your choice is valid (ie not `None`).

# Setting Values

To set values in PyEPICS use `pv.put(the_value)` or assign a value to the `pv.value` attribute.

For example:

```python
pv = PV('SOME_MOTOR:SP')
pv.put(8.2)
pv.value = 7.3
```

For enum PVs, you can assign the value using either the state number or state string.

### Challenge:

1. Write values to "`PREFIX + ':SETPOINT'`" and observe the value changing at `http://pythonworkshop.staff.synchrotron.org.au/iocs/<your-ioc-prefix>`.
2. Try driving "`PREFIX + ':SETPOINT'`" beyond its "drive high" limit of 10.
3. Set the "`PREFIX + ':ALERT'`" enum PV to one of the states from the `enum_strs`.

# Callback functions

Callback functions are used to receive notifications when PVs state changes. For example:

* When the PV value changes
* When the PV connects or disconnects
* When the alarm severity changes

## Value callbacks

To register a value callback you define a regular Python function that accepts a long list of
arguments that are [listed in the PyEPICS documentation](http://cars9.uchicago.edu/software/python/pyepics3/pv.html#user-supplied-callback-functions). Fortunately
Python has a convenient syntax for catching all the arguments that you aren't really interested in
into a dictonary. You simply prefix the final argument in your function definition with two asterisks:

In [ ]:
def example_function(x, **other_arguments_i_dont_care_about):
    print('x:', x)
    print('everything else:', other_arguments_i_dont_care_about)
    
example_function(x=1, y=2, z=3)

This enables us to define a callback function such as:

In [ ]:
import ipywidgets
from IPython.display import display

textbox = ipywidgets.Textarea()
display(textbox)

def handle_value_update(pvname, value, **other_params):
    #print(pvname, value, flush=True)
    textbox.value = '{}\n{}\n{}'.format(pvname, value, other_params['severity'])

#### Add the callback to the PV

In [ ]:
pv = PV(PREFIX + ':TEMPERATURE', callback=handle_value_update)

#### Remove callbacks

In [ ]:
pv.clear_callbacks()

### What *not* to do inside callback functions

If you try and do any slow operation or certain EPICS operations inside a callback function you
can block PyEPICS from processing other callbacks.

## Running a task from a callback:
## The *wrong* way

Execute the following block and observe that whenever the temperature callback is triggered
it blocks `fast_random_callback` from running.

In [ ]:
from time import sleep

fast_random_textbox = ipywidgets.Text()
temperature_textbox = ipywidgets.Text()
display(fast_random_textbox, temperature_textbox)

def fast_random_callback(value, **kwargs):
    fast_random_textbox.value = str(value)
    
def temperature_callback(char_value, **kwargs):
    temperature_textbox.value = 'RUNNING SLOW TASK'
    sleep(.4)
    temperature_textbox.value = 'DONE!'

fast_random_pv = PV(PREFIX + ':FAST_RANDOM', callback=fast_random_callback)
temperature_pv = PV(PREFIX + ':TEMPERATURE', callback=temperature_callback)

In [ ]:
fast_random_pv.clear_callbacks()
temperature_pv.clear_callbacks()

# Options

When you recieve a callback you should delegate the work to a seperate thread.

You can either:

* Spawn a new thread from the callback.
* Trigger processing on a long running (daemon) thread using a queue or signal. See:

    * `queue` package:
    https://docs.python.org/3/library/queue.html
    * `threading` package: https://docs.python.org/3/library/threading.html

## Running a task from a callback: The right way!

In the next cell we use the `threading` library to run the slow task inside a seperate thread so
it doesn't block the PyEPICS callbacks.

### `threading.Event`

* Enables signalling between threads
* Calling `event.wait()` will pause execution until the event is triggered
* Calling `event.set()` signals the event has happened, allowing any threads that were waiting for it to run

In [ ]:
import threading

DO_SLOW_WORK = threading.Event()

fast_random_textbox = ipywidgets.Text()
temperature_textbox = ipywidgets.Text()
display(fast_random_textbox, temperature_textbox)

def worker():
    while True:
        DO_SLOW_WORK.wait()  # <= Blocks until event is triggered
        temperature_textbox.value = 'RUNNING SLOW TASK'
        sleep(.4)
        temperature_textbox.value = 'DONE!'
        DO_SLOW_WORK.clear()

threading.Thread(target=worker, daemon=True).start()  # start the worker function in a different thread

def fast_random_callback(value, **kwargs):
    fast_random_textbox.value = str(value)
    
def temperature_callback(**kwargs):
    DO_SLOW_WORK.set()  # delegate to worker

fast_random_pv = PV(PREFIX + ':FAST_RANDOM', callback=fast_random_callback)
temperature_pv = PV(PREFIX + ':TEMPERATURE', callback=temperature_callback)

In [ ]:
fast_random_pv.clear_callbacks()
temperature_pv.clear_callbacks()

## Devices

The PyEPICS `Device` class provides a structure for grouping related PVs. For example, a motor may have:

* the requested position
* the actual position
* the requested speed
* the current speed
* ...

All of these can be stored and accessed through a single `Device`.

In [ ]:
demo_device = epics.Device(prefix=PREFIX + ':')
print(demo_device)

In [ ]:
demo_device.TEMPERATURE

In [ ]:
demo_device.ALERT

In [ ]:
demo_device.get('ALERT', as_string=True)

In [ ]:
demo_device.SETPOINT = 8

# Under the covers

In [ ]:
print(demo_device)

In [ ]:
print(demo_device._pvs)

## Defensive programming with the Device class

When you request any attribute on your device, PyEPICS assumes it is a PV and tries to connect to it.
If a PV doesn't exist with the name PyEPICS will eventually time out but this could lead to unwanted
behavior.

We recommend always being explicit about what PVs you want your device to support by
specifying an aliases dictionary and **`mutable=False`**:

In [ ]:
safe_demo_device = epics.Device(prefix=PREFIX + ':',
                                aliases={'target': 'SETPOINT',
                                         'value': 'READBACK',
                                         'temperature': 'TEMPERATURE'},
                                mutable=False)
print(safe_demo_device)

In [ ]:
safe_demo_device.target = 1.5

In [ ]:
safe_demo_device.invalid_name  # Instantly raises an exception rather than trying to make a PV

### Challenges:

1. Extend the device to have attributes for the other PVs listed on `http://pythonworkshop.staff.synchrotron.org.au/iocs/<your-ioc-prefix>`.
2. Use the `Device.save_state()` to capture the state of all PVs into a dictionary. Store this in a variable.
3. Change the devices setpoint and then restore the state with the `Device.restore_state()` method.
4. Save the devices state to a text file with `Device.write_state()`.

## Subclassing `Device`

Extra functionality can be added to the `Device` class by subclassing it. For example:

In [ ]:
class MyDevice(epics.Device):
    def __init__(self, aliases=None, mutable=False, **kws):
        if aliases is None:
            aliases = {}
        aliases.update({'target': 'SETPOINT',
                        'readback': 'READBACK',
                        'text_array': 'LONG_STRING'})
        super().__init__(aliases=aliases, mutable=mutable, **kws)
    
    def go_to_setpoint(self, setpoint):
        self.target = setpoint
        while abs(self.readback - setpoint) > .1:
            print('Moving...', flush=True)
            sleep(.5)


my_device = MyDevice(prefix=PREFIX + ':')

In [ ]:
my_device.go_to_setpoint(1.9)

### Challenge:

Add a property to the class to get the `LONG_STRING` PVs using `my_device.text`. Hint: you can call the `.get()` method on a PV inside a device using `self.get('alias', ...)`.

# Other features

* Alarm class http://pyepics.github.io/pyepics/alarm.html
* wxPython Widgets http://pyepics.github.io/pyepics/wx.html
* autosave http://pyepics.github.io/pyepics/autosave.html
* PyEPICS Applications http://pyepics.github.io/epicsapps/

# Final thoughts

* PyEPICS documentation is very good:
  https://pyepics.github.io/pyepics/
* The PyEPICS code is very readable - dive in at:
  https://github.com/pyepics/pyepics
* Don't be afraid to report bugs!
* Recommendations available on the [Confluence PyEPICS page](https://confluence.synchrotron.org.au/display/LANG/PyEPICS)